# Chains

> Chains-based functions for PDFs.

In [ ]:
# | default_exp pdf.chains

In [ ]:
# | export

from langchain_ray.imports import *
from langchain_ray.chains import *
from langchain_ray.pdf.utils import *

In [ ]:
# | export


def pdf_docs_chain(
    chunk_size=200, chunk_overlap=20, verbose=False, input_key="pdf_folder", output_key="df"
):
    "Chain for loading and splitting PDFs into Documents."
    pdf_chain = transform_chain(create_pdf_df, input_key=input_key)
    docs_chain = transform_chain(
        df_pdf_docs,
        transform_kwargs={"chunk_size": chunk_size, "chunk_overlap": chunk_overlap},
    )
    return SimpleSequentialChain(
        chains=[pdf_chain, docs_chain],
        input_key=input_key,
        output_key=output_key,
        verbose=verbose,
    )


def pdf_cats_chain(cats_model, input_key="df", output_key="df"):
    "Chain for adding categories to PDF Documents."
    return transform_chain(
        df_docs_cat,
        input_key=input_key,
        output_key=output_key,
        transform_kwargs={"cats_model": cats_model},
    )


def pdf_ems_chain(ems_model, ems_folder, input_key="df", output_key="df"):
    "Chain for adding embeddings to a PDF Documents DataFrame."
    transform_chain(
        df_docs_ems,
        input_key=input_key,
        output_key=output_key,
        transform_kwargs={
            "ems_model": ems_model,
            "ems_folder": ems_folder,
        },
    )


def pdf_faiss_chain(ems_model, index_folder, index_name, input_key="df", output_key="df"):
    "Chain for adding PDF Documents to a FAISS index."
    return transform_chain(
        df_to_faiss,
        input_key=input_key,
        output_key=output_key,
        transform_kwargs={
            "ems_model": ems_model,
            "index_folder": index_folder,
            "index_name": index_name,
        },
    )


def pdf_index_chain(
    ems_model,  # The SentenceTransformer model to use for vectorestore embeddings.
    index_folder,  # The folder to store the FAISS index.
    index_name,  # The name of the FAISS index.
    input_key="pdf_folder",  # The input key for the PDF folder.
    output_key="df",  # The output key for the final DataFrame.
    chunk_size=200,  # The number of characters per Document.
    chunk_overlap=20,  # The number of characters to overlap between Documents.
    docs_block_size=1500,  # The number of Documents to process in a single Ray task.
    cats_model=None,  # The HuggingFace model to use for categorization.
    ems_chain_model=None,  # The SentenceTransformer model to use for chain embeddings.
    ems_folder=None,  # The folder to store the embeddings.
    verbose=False,
):
    """
    Chain for adding PDF documents to a FAISS index.
    It will be divided and distributed into multiple indexes using Ray if there are more documents than `docs_block_size`.
    """
    chain1 = pdf_docs_chain(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, input_key=input_key
    )
    index_chains = []
    if cats_model is not None:
        cats_chain = pdf_cats_chain(cats_model)
        index_chains.append(cats_chain)
    if ems_folder is not None and ems_chain_model is not None:
        ems_chain = pdf_ems_chain(ems_chain_model, ems_folder)
        index_chains.append(ems_chain)

    faiss_chain = pdf_faiss_chain(ems_model, index_folder, index_name)
    index_chains.append(faiss_chain)
    chain2 = ray_chain(
        SimpleSequentialChain(chains=index_chains),
        block_size=docs_block_size,
        cuda=True,
    )
    return SimpleSequentialChain(
        chains=[chain1, chain2], input_key=input_key, output_key=output_key, verbose=verbose
    )


def index_query_chain(
    ems_model,  # The SentenceTransformer model to use for vectorestore embeddings.
    index_folder,  # The folder with the FAISS indexes.
    index_name,  # The name of the FAISS index.
    input_key="query",  # The input key for the query.
    output_key="search_results",  # The output key for the search results.
    k=2,  # The number of results to return.
    block_size=10,  # The number of indexes to process in a single Ray task.
    verbose=False,
):
    "Chain for querying the FAISS indexes in `index_folder` with `index_name`."
    q_df_chain = transform_chain(
        create_idx_q_df,
        input_key=input_key,
        transform_kwargs={"index_folder": index_folder, "index_name": index_name},
    )

    search_chain = transform_chain(
        lambda df: df.apply(df_search_faiss, axis=1, ems_model=ems_model, k=k),
    )

    res_chain = transform_chain(
        lambda df: sorted(flatten_list(df.results), key=lambda x: x[1]),
    )

    return ray_chain(
        SimpleSequentialChain(
            chains=[q_df_chain, search_chain, res_chain],
            input_key=input_key,
            output_key=output_key,
            verbose=verbose,
        ),
        block_size=block_size,
        cuda=True,
    )

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()